In [2]:
import sys
import os
import math
import argparse

sys.path.append('cryoem/')
sys.path.append('cryoem/util')

import numpy as np
import pandas as pd

import matplotlib as mpl
import matplotlib.pyplot as plt

mpl.rcParams['figure.dpi'] = 100
plt.style.use(['dark_background'])

In [3]:
def read_starfile(path):
	# Read the theoretical starfile
	# We only want (1-indexed): 2 (psi), 3 (phi), 4 (theta), 12 (originX), 13 (originY) 
	# BEWARE skiprows, starfile header lengths may vary
	theoretical = pd.read_csv(path, delim_whitespace=True, header=None, skiprows=21, low_memory=False)
	theoretical = theoretical[theoretical.columns[[1, 2, 3, 11, 12]]]
	theoretical.columns = [ 'psi', 'phi', 'theta', 'shiftX', 'shiftY']
	theoretical = theoretical.astype(float)
	theoretical['quaternion'] = theoretical.apply(lambda row: euler2quat(row.phi*np.pi/180, row.theta*np.pi/180, row.psi*np.pi/180), axis=1)
	return theoretical

def read_parfile(path):
	# Read the experimental parfile
	# BEWARE dropping last two rows.
	experimental = pd.read_csv(path, delim_whitespace=True, low_memory=False)
	experimental = experimental[experimental.columns[[1, 3, 2, 4, 5]]]
	experimental.columns = [ 'psi', 'phi', 'theta', 'shiftX', 'shiftY']
	experimental.drop(experimental.tail(2).index,inplace=True)
	experimental = experimental.astype(float)
	experimental['quaternion'] = experimental.apply(lambda row: euler2quat(row.phi*np.pi/180, row.theta*np.pi/180, row.psi*np.pi/180), axis=1)
	return experimental

def euler2quat(alpha, beta, gamma):
	ha, hb, hg = alpha / 2, beta / 2, gamma / 2
	ha_p_hg = ha + hg
	hg_m_ha = hg - ha
	q = np.array([np.cos(ha_p_hg) * np.cos(hb),
				  np.sin(hg_m_ha) * np.sin(hb),
				  np.cos(hg_m_ha) * np.sin(hb),
				  np.sin(ha_p_hg) * np.cos(hb)])
	return q

# Quaternion to Euler Angles (from https://github.com/asarnow/pyem geom.py)
def quat2euler(q):
	ha1 = np.arctan2(q[1], q[2])
	ha2 = np.arctan2(q[3], q[0])
	alpha = ha2 - ha1  # np.arctan2(r21/r20)
	beta = 2 * np.arccos(np.sqrt(q[0]**2 + q[3]**2))  # np.arccos*r33
	gamma = ha1 + ha2  # np.arctan2(r12/-r02)
	return alpha, beta, gamma

# Angular distance between two quaternions

def quatInverse(q):
	d = q[0]**2 + q[1]**2 + q[2]**2 + q[3]**2
	d = 1
	return [q[0]/d, -q[1]/d, -q[2]/d, -q[3]/d]

def quatConj(q):
	return [q[0],-q[1],-q[2],-q[3]]

def quatDist(a,b):
	# Check to verify that quaternions are unit lengths
	assert abs(math.sqrt(a[0]**2+a[1]**2+a[2]**2+a[3]**2)-1)<.001,"a is not a unit quaternion"
	assert abs(math.sqrt(b[0]**2+b[1]**2+b[2]**2+b[3]**2)-1)<.001,"b is not a unit quaternion"

	# # # Compute distance
	s = a[0]*b[0] + a[1]*b[1] + a[2]*b[2] + a[3]*b[3]
	s = round(s,4)

	s = 2*(s**2)-1
	other = (np.arccos(s))*180/np.pi
	return other

	# z = a*quatConj(b)
	# theta = 2*np.arccos(z[0])
	# o = abs(theta*180/np.pi - 180)
	# print("Method2: %f"%o)
	# return abs(theta*180/np.pi - 180)



	# a = quatInverse(a)

	# s = [a[0]*b[0] , a[1]*b[1] , a[2]*b[2] , a[3]*b[3]]

	# l = math.sqrt(s[0]**2+s[1]**2+s[2]**2+s[3]**2)

	# o = 2*math.atan2(l,s[3])

	# return (o*180/np.pi)

	# # print("s = %f" % s)
	# # assert s <= 1, "product greater than 1"
	# # assert s >= -1, "product less than -1"

	# try:
	# 	output = 2*np.arccos(s)*180/np.pi
	# 	return output
	# except:
	# 	e = sys.exc_info()[0]
	# 	print("Quat Distance Error: %s" % e)
	# 	return 50

# Given two ordered lists of quaternions, compute distances between each angle
def computeAngleErrors(A, B):
	qq = zip(A, B)
	errors = []
	for i,v in enumerate(qq):
		dist = quatDist(v[0],v[1])
		errors.append((dist))

	return errors

# A and B are (x,y) tuples
def euclideanDistance(A,B):
	return math.sqrt((A[0]-B[0])**2 + (A[1]-B[1])**2)

# theoretical and experimental are arrays of (x,y) tuples
def computeShiftErrors(theoretical, experimental):
	ab = zip(theoretical, experimental)
	errors = []
	for i, v in enumerate(ab):
		dist = euclideanDistance(v[0],v[1])
		errors.append(dist)
	return errors

def mean(array):
	return sum(array)/len(array)